# Demo Peg Keeping Logic

One of the agents in our simulation is the `Keeper`. The `Keeper` calls the `update` function on all Peg Keepers if doing so exceeds their profitability threshold (default 0). This notebook demos the `Keeper`'s `update` method.

In [1]:
from src.sim.scenario import Scenario

In [2]:
def increment_timestamps(aggregator, pks):
    ts = aggregator._block_timestamp + 60 * 60  # one hour
    for pk in pks:
        pk._increment_timestamp(ts)
        pk.POOL._increment_timestamp(ts)
        assert pk.last_change != pk._block_timestamp
    aggregator._increment_timestamp(ts)
    print(f"New Aggregator Price: {aggregator.price() / 1e18}")

In [3]:
# Generate markets and prices
scenario = Scenario("baseline")
scenario.prepare_for_run()

[INFO][03:30:15][src.sim.scenario]-963133: Using 711360 1Inch quotes.
[INFO][03:30:15][src.sim.scenario]-963133: Fetching sim_market from subgraph.
[INFO][03:30:18][src.utils.poolgraph]-963133: Found 20 valid cycles of length 3.


In [4]:
aggregator = scenario.aggregator
aggregator.price() / 1e18

0.9991866780961878

In [5]:
pks = scenario.peg_keepers
keeper = scenario.keeper
profit, count = keeper.update(pks)
print(f"Profit: {profit}, Count: {count}")

[INFO][03:30:18][src.agents.keeper]-963133: Not updating crvUSD/TUSD Peg Keeper.
[INFO][03:30:18][src.agents.keeper]-963133: Not updating crvUSD/USDP Peg Keeper.
[INFO][03:30:18][src.agents.keeper]-963133: Not updating crvUSD/USDC Peg Keeper.
[INFO][03:30:18][src.agents.keeper]-963133: Not updating crvUSD/USDT Peg Keeper.


Profit: 0.0, Count: 0


In all likelihood, no updates occured even though the pools are not balanced. This is because the balance of crvUSD exceeds the balance of the peg stablecoin (e.g. USDC), but the `PegKeeper` doesn't have any debt to withdraw. To test the `update` functionality, we trade against the stableswap pools such that the crvUSD balance is less than the peg stablecoin balance.

In [10]:
for pk in pks:
    spool = pk.POOL
    print(spool.name)

    normalized_balances = [b * r / 1e18 for b, r in zip(spool.balances, spool.rates)]
    print(f"Normalized balances before trade: {normalized_balances}")

    diff = normalized_balances[pk.I] - normalized_balances[pk.I ^ 1]
    if diff < 1:
        continue

    diff = int(diff * 1e18 / spool.rates[pk.I ^ 1])  # convert to peg coin units
    print(
        f"Swapping in {diff} {spool.coins[pk.I^1].symbol} for {spool.coins[pk.I].symbol}"
    )

    amt_in, amt_out, fees = spool.trade(pk.I ^ 1, pk.I, diff)
    assert amt_in == diff
    print(f"Received: {amt_out}, Fees: {fees}")
    normalized_balances = [b * r / 1e18 for b, r in zip(spool.balances, spool.rates)]
    print(f"Normalized balances after trade: {normalized_balances}")

    print()

Curve.fi Factory Plain Pool: crvUSD/TUSD
Normalized balances before trade: [2.487111603121784e+24, 1.9150059024137788e+24]
Curve.fi Factory Plain Pool: crvUSD/USDP
Normalized balances before trade: [1.6294304079144517e+24, 2.243271069396673e+24]
Swapping in 613840661482221315555328 USDP for crvUSD
Received: 613779277416073093484167, Fees: 61384066148222131561
Normalized balances after trade: [2.243271069396673e+24, 1.629461099947526e+24]

Curve.fi Factory Plain Pool: crvUSD/USDC
Normalized balances before trade: [1.3929085025007998e+25, 2.074047893642343e+25]
Swapping in 6811393911415 USDC for crvUSD
Received: 6810712772023858866331488, Fees: 681139391141500036636
Normalized balances after trade: [2.0740478936423002e+25, 1.3929425594704002e+25]

Curve.fi Factory Plain Pool: crvUSD/USDT
Normalized balances before trade: [1.3716983508907999e+25, 2.0259484621668413e+25]
Swapping in 6542501112760 USDT for crvUSD
Received: 6541846862648724343633905, Fees: 654250111276000034366
Normalized ba

In [11]:
# Increment timestamps to update aggregator price
# AND pk timestamps
increment_timestamps(aggregator, pks)

New Aggregator Price: 1.0007895361659047


In [12]:
profit, count = keeper.update(pks)
print(f"Profit: {profit / 1e18}, Count: {count}")

[INFO][03:32:50][src.agents.keeper]-963133: Updating crvUSD/TUSD Peg Keeper with profit 4.
[INFO][03:32:50][src.agents.keeper]-963133: Updating crvUSD/USDP Peg Keeper with profit 10.
[INFO][03:32:50][src.agents.keeper]-963133: Updating crvUSD/USDC Peg Keeper with profit 104.
[INFO][03:32:50][src.agents.keeper]-963133: Updating crvUSD/USDT Peg Keeper with profit 191.


Profit: 190.7624801107767, Count: 4


In [13]:
for pk in pks:
    spool = pk.POOL
    print(spool.name)

    normalized_balances = [b * r / 1e18 for b, r in zip(spool.balances, spool.rates)]
    print(f"Normalized balances before trade: {normalized_balances}")
    print()

Curve.fi Factory Plain Pool: crvUSD/TUSD
Normalized balances before trade: [2.487109986545066e+24, 2.0294254267454297e+24]

Curve.fi Factory Plain Pool: crvUSD/USDP
Normalized balances before trade: [2.24326929105439e+24, 1.7522213165349084e+24]

Curve.fi Factory Plain Pool: crvUSD/USDC
Normalized balances before trade: [2.0740458554636e+25, 1.5291615896307565e+25]

Curve.fi Factory Plain Pool: crvUSD/USDT
Normalized balances before trade: [2.0259465108621e+25, 1.5025725932547138e+25]



In [15]:
# Increment timestamps to update aggregator price
# AND pk timestamps
increment_timestamps(aggregator, pks)

New Aggregator Price: 1.0006077585398148


In [16]:
# We can update again! Recall that the PK can only
# deposit/withdraw 20% of the pool's imbalance at a time.
profit, count = keeper.update(pks)
print(f"Profit: {profit / 1e18}, Count: {count}")

[INFO][03:33:09][src.agents.keeper]-963133: Updating crvUSD/TUSD Peg Keeper with profit 3.
[INFO][03:33:09][src.agents.keeper]-963133: Updating crvUSD/USDP Peg Keeper with profit 6.
[INFO][03:33:09][src.agents.keeper]-963133: Updating crvUSD/USDC Peg Keeper with profit 59.
[INFO][03:33:09][src.agents.keeper]-963133: Updating crvUSD/USDT Peg Keeper with profit 108.


Profit: 107.8561267863586, Count: 4


In [17]:
# Increment timestamps to update aggregator price
# AND pk timestamps
increment_timestamps(aggregator, pks)

New Aggregator Price: 1.0004643008580523
